In [2]:
import pandas as pd 
import numpy as np
import function
import matplotlib.pyplot as plt

In [3]:
train_df = pd.read_csv("./train.csv")
print(train_df.columns)
print(train_df.shape)

Index(['image_id', 'class_name', 'class_id', 'rad_id', 'x_min', 'y_min',
       'x_max', 'y_max'],
      dtype='object')
(67914, 8)


In [4]:
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN


In [5]:
image_id = train_df["image_id"]
class_name =  train_df["class_name"]
class_id = train_df["class_id"] 
rad_id = train_df["rad_id"]
x_min = train_df["x_min"]
y_min = train_df["y_min"]
x_max = train_df["x_max"]
y_max = train_df["y_max"]

train_df = train_df.fillna(0)
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,0.0,0.0,0.0,0.0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,0.0,0.0,0.0,0.0
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,0.0,0.0,0.0,0.0


In [6]:
img_len = len(train_df["image_id"].value_counts())
print(img_len)

img_name_list = train_df["image_id"].value_counts().index.tolist()
print(len(img_name_list))
print(len(set(img_name_list)))

15000
15000
15000


In [7]:
limit = 0.5
idx_list = [] # iou 이상인 index끼리 담아 놓은

for img_name in img_name_list:
    indexes = train_df[train_df["image_id"] == img_name].index
    for i in range(len(indexes)-1):
        for j in range(i+1,len(indexes)):
            rect1 = (x_min[indexes[i]], y_min[indexes[i]], x_max[indexes[i]], y_max[indexes[i]])
            rect2 = (x_min[indexes[j]], y_min[indexes[j]], x_max[indexes[j]], y_max[indexes[j]])
            iou = function.get_iou(rect1, rect2)
            if(iou >= limit):
                print(iou)
                idx_list.append((indexes[i], indexes[j])) 
                

8334
0.5189151876818334
0.5582351728632305
0.7043580682402443
0.7472613784814741
0.7849463796008094
0.671125489116944
0.7760496150962675
0.5469908647232654
0.6095933077730554
0.6424454061929524
0.6658814799283874
0.6995157596908269
0.6675736960610262
0.5804582829782318
0.6828034567355041
0.7318183576715671
0.5078505098965531
0.6589373217933144
0.999999999909223
0.6589373217933144
0.675090413834434
0.7208882364957956
0.8014091537056645
0.7170377228635374
0.6689113738669457
0.6632939853218535
0.9460916410172114
0.8737627531355465
0.8647503327463762
0.8746892980185647
0.5633232073956387
0.5655974106643947
0.7174825869674125
0.6672763702157042
0.8185819686001136
0.9510855553014426
0.8300771963616975
0.7172763796579128
0.6576105885496327
0.6414081144714948
0.8503649634690121
0.6531868284426968
0.6519936293050069
0.6831719505677347
0.5135654548846845
0.9999999995704099
0.8336314103649523
0.643979057521381
0.6465857636755
0.9999999997886013
0.6258583671560347
0.5727351566179594
0.623548760416

In [8]:
print(len(idx_list))
idx_list[0:10]

26472


[(757, 37684),
 (1906, 3318),
 (1906, 41550),
 (2087, 50893),
 (2489, 49434),
 (3318, 41550),
 (12042, 66294),
 (12234, 13136),
 (14463, 59265),
 (16073, 63260)]

In [9]:
overlap_df = pd.DataFrame(index=img_name_list, columns=['overlap_class_name', 'overlap_class_id', 'overlap_rad_id', 'overlap_index'])
print(overlap_df.shape)
overlap_df.replace(np.NAN, "", inplace=True)
overlap_df

(15000, 4)


,overlap_class_name,overlap_class_id,overlap_rad_id,overlap_index
03e6ecfa6f6fb33dfeac6ca4f9b459c9,,,,
fa109c087e46fe1ea27e48ce6d154d2f,,,,
e31be972e181987a8600a8700c1ebe88,,,,
3a302fbbbf3364aa1a7731b59e6b98ec,,,,
6d5acf3f8a973a26844d617fffe72998,,,,
...,...,...,...,...
bab58da5c7d82e57bd63dfa1d6e698a4,,,,
37f1cd607b0fe559074fdad2c5829887,,,,
26892c0317a9d478211fae77c464f62d,,,,
94c681d8fed39d275b4a0db2a914fb6d,,,,


In [10]:
for idx in idx_list: # ( a, b )
    df_idx1 = idx[0]
    df_idx2 = idx[1]
    ocn = overlap_df.loc[[image_id[df_idx1]]]["overlap_class_name"][0] +  str(train_df.loc[df_idx1][1]) + '-' + str(train_df.loc[df_idx2][1]) + '-'
    oci = overlap_df.loc[[image_id[df_idx1]]]["overlap_class_id"][0] +  str(train_df.loc[df_idx1][2]) + '-' + str(train_df.loc[df_idx2][2]) + '-'
    ori = overlap_df.loc[[image_id[df_idx1]]]["overlap_rad_id"][0] +  str(train_df.loc[df_idx1][3]) + '-' + str(train_df.loc[df_idx2][3]) + '-'
    oi = overlap_df.loc[[image_id[df_idx1]]]["overlap_index"][0] +  str(df_idx1) + '-' + str(df_idx2) + '-'
    overlap_df.loc[image_id[df_idx1]] = [ocn, oci, ori, oi]
overlap_df

,overlap_class_name,overlap_class_id,overlap_rad_id,overlap_index
03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-5-8-8-8-8-8-8-8-8-8-8-8-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R10-R9-R8-R9-R8-R9-R8-R9-R...,757-37684-1906-3318-1906-41550-2087-50893-2489...
fa109c087e46fe1ea27e48ce6d154d2f,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-8-8-8-8-8-7-8-7-8-7-8-8-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R10-R8-R10-R9-R8-R9-R10-R1...,7301-51569-7627-10763-7627-45183-10088-20915-1...
e31be972e181987a8600a8700c1ebe88,Nodule/Mass-Nodule/Mass-Nodule/Mass-Calcificat...,8-8-8-2-11-2-11-2-11-11-11-2-11-11-11-2-2-11-2...,R10-R9-R10-R8-R8-R8-R10-R10-R10-R8-R10-R9-R10-...,496-13033-496-52024-606-65305-1156-17557-1156-...
3a302fbbbf3364aa1a7731b59e6b98ec,Nodule/Mass-Nodule/Mass-Nodule/Mass-Lung Opaci...,8-8-8-7-8-7-8-8-8-7-8-7-8-7-10-10-10-10-8-7-8-...,R9-R8-R9-R10-R9-R9-R9-R10-R9-R10-R9-R9-R10-R10...,237-20856-237-38643-237-56033-237-59273-1432-2...
6d5acf3f8a973a26844d617fffe72998,Nodule/Mass-Nodule/Mass-Lung Opacity-ILD-Lung ...,8-8-7-5-7-5-8-8-8-8-9-9-5-5-0-0-0-0-5-7-5-5-8-...,R8-R9-R9-R9-R9-R8-R8-R9-R8-R9-R8-R9-R9-R8-R10-...,1550-4753-4673-9517-4673-21775-5263-61141-6208...
...,...,...,...,...
bab58da5c7d82e57bd63dfa1d6e698a4,,,,
37f1cd607b0fe559074fdad2c5829887,,,,
26892c0317a9d478211fae77c464f62d,,,,
94c681d8fed39d275b4a0db2a914fb6d,,,,


In [14]:
overlap_df.replace("", np.NAN, inplace=True)

In [15]:
overlap_df

,overlap_class_name,overlap_class_id,overlap_rad_id,overlap_index
03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-5-8-8-8-8-8-8-8-8-8-8-8-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R10-R9-R8-R9-R8-R9-R8-R9-R...,757-37684-1906-3318-1906-41550-2087-50893-2489...
fa109c087e46fe1ea27e48ce6d154d2f,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-8-8-8-8-8-7-8-7-8-7-8-8-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R10-R8-R10-R9-R8-R9-R10-R1...,7301-51569-7627-10763-7627-45183-10088-20915-1...
e31be972e181987a8600a8700c1ebe88,Nodule/Mass-Nodule/Mass-Nodule/Mass-Calcificat...,8-8-8-2-11-2-11-2-11-11-11-2-11-11-11-2-2-11-2...,R10-R9-R10-R8-R8-R8-R10-R10-R10-R8-R10-R9-R10-...,496-13033-496-52024-606-65305-1156-17557-1156-...
3a302fbbbf3364aa1a7731b59e6b98ec,Nodule/Mass-Nodule/Mass-Nodule/Mass-Lung Opaci...,8-8-8-7-8-7-8-8-8-7-8-7-8-7-10-10-10-10-8-7-8-...,R9-R8-R9-R10-R9-R9-R9-R10-R9-R10-R9-R9-R10-R10...,237-20856-237-38643-237-56033-237-59273-1432-2...
6d5acf3f8a973a26844d617fffe72998,Nodule/Mass-Nodule/Mass-Lung Opacity-ILD-Lung ...,8-8-7-5-7-5-8-8-8-8-9-9-5-5-0-0-0-0-5-7-5-5-8-...,R8-R9-R9-R9-R9-R8-R8-R9-R8-R9-R8-R9-R9-R8-R10-...,1550-4753-4673-9517-4673-21775-5263-61141-6208...
...,...,...,...,...
bab58da5c7d82e57bd63dfa1d6e698a4,NaN,NaN,NaN,NaN
37f1cd607b0fe559074fdad2c5829887,NaN,NaN,NaN,NaN
26892c0317a9d478211fae77c464f62d,NaN,NaN,NaN,NaN
94c681d8fed39d275b4a0db2a914fb6d,NaN,NaN,NaN,NaN


In [16]:
print(overlap_df.isnull())
overlap_df.isnull().sum()
# 15000 - 10690 = 4310개 이미지에 겹치는 부분 있음

                                  overlap_class_name  overlap_class_id  \
03e6ecfa6f6fb33dfeac6ca4f9b459c9               False             False   
fa109c087e46fe1ea27e48ce6d154d2f               False             False   
e31be972e181987a8600a8700c1ebe88               False             False   
3a302fbbbf3364aa1a7731b59e6b98ec               False             False   
6d5acf3f8a973a26844d617fffe72998               False             False   
...                                              ...               ...   
bab58da5c7d82e57bd63dfa1d6e698a4                True              True   
37f1cd607b0fe559074fdad2c5829887                True              True   
26892c0317a9d478211fae77c464f62d                True              True   
94c681d8fed39d275b4a0db2a914fb6d                True              True   
d8f5c5ba4ef51f94757d91991244f1b9                True              True   

                                  overlap_rad_id  overlap_index  
03e6ecfa6f6fb33dfeac6ca4f9b459c9           Fa

overlap_class_name    10690
overlap_class_id      10690
overlap_rad_id        10690
overlap_index         10690
dtype: int64

In [17]:
# 사진
import cv2
img = function.read_xray('./dataset/03e6ecfa6f6fb33dfeac6ca4f9b459c9.dicom')
img = cv2.merge((img, img, img))

tlist = overlap_df.loc["03e6ecfa6f6fb33dfeac6ca4f9b459c9"]["overlap_index"].split('-')
tlist = tlist[0:-1]
tlist = [int(i) for i in tlist]
tlist = set(tlist)

all_list = image_id[image_id == "03e6ecfa6f6fb33dfeac6ca4f9b459c9"].index
all_list = set(all_list)
diff_list = all_list.difference(tlist)


for index in tlist:
    cv2.rectangle(img, (int(x_min[index]), int(y_min[index])), (int(x_max[index]), int(y_max[index])), function.red, 3) 
    cv2.putText(img, str(class_name[index]), (int(x_min[index]), int(y_min[index]-3)),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255) , 2)

for index in diff_list:
    cv2.rectangle(img, (int(x_min[index]), int(y_min[index])), (int(x_max[index]), int(y_max[index])), function.blue, 3) 
    cv2.putText(img, str(class_name[index]), (int(x_min[index]), int(y_min[index]-3)),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255) , 2)

img = function.FitToWindowSize(img)
cv2.imshow("", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
overlap_df = overlap_df.dropna()
overlap_df.shape

(4310, 4)

In [23]:
overlap_index_list = overlap_df.index

In [109]:
chk_list = []
for index in overlap_index_list:
    index_group = overlap_df.loc[index].overlap_index
    index_set = index_group.split('-')[0:-1]
    index_set = list(set(index_set))
    for i in range(len(index_set)-1):
        x = int(index_set[i])
        txt = class_name[x]
        print(f'x, {x} {txt}')
        for j in range(i+1, len(index_set)):
            y = int(index_set[j])
            print(f'y, {y} {class_name[y]}')
            if txt not in class_name[y]:  
                txt_result = txt + ' + ' + class_name[y]
        print(f'final {txt_result}')
        if txt != txt_result:
            chk_list.append(txt_result)

c enlargement
y, 6682 Aortic enlargement
y, 8082 Aortic enlargement
final Nodule/Mass + Calcification
x, 6682 Aortic enlargement
y, 8082 Aortic enlargement
final Nodule/Mass + Calcification
x, 21256 Aortic enlargement
y, 59952 Aortic enlargement
y, 53313 Aortic enlargement
final Nodule/Mass + Calcification
x, 59952 Aortic enlargement
y, 53313 Aortic enlargement
final Nodule/Mass + Calcification
x, 48912 Pleural thickening
y, 6075 Pleural thickening
final Nodule/Mass + Calcification
x, 52740 Cardiomegaly
y, 30625 Cardiomegaly
y, 58569 Cardiomegaly
final Nodule/Mass + Calcification
x, 30625 Cardiomegaly
y, 58569 Cardiomegaly
final Nodule/Mass + Calcification
x, 4342 Aortic enlargement
y, 5398 Aortic enlargement
y, 63330 Aortic enlargement
final Nodule/Mass + Calcification
x, 5398 Aortic enlargement
y, 63330 Aortic enlargement
final Nodule/Mass + Calcification
x, 32364 Aortic enlargement
y, 34501 Aortic enlargement
y, 59554 Aortic enlargement
final Nodule/Mass + Calcification
x, 34501 Aor

In [110]:
chk_list[0:10]

['Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'ILD + Nodule/Mass',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD']

In [111]:
for i in range(len(chk_list)):
    tmp = sorted(chk_list[i].split(' + '))
    result = tmp[0]
    for j in range(1,len(tmp)):
        result = result + ' + ' + tmp[j] 
    chk_list[i] = result
chk_list[0:10]

['ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass']

In [112]:
from collections import Counter

result = Counter(chk_list)
print(result)

for key in result:
    print(key, result[key])

Counter({'Aortic enlargement + Cardiomegaly': 5326, 'Pleural effusion + Pleural thickening': 1253, 'Pleural thickening + Pulmonary fibrosis': 847, 'Infiltration + Pulmonary fibrosis': 698, 'Aortic enlargement + Pleural thickening': 663, 'Aortic enlargement + Pulmonary fibrosis': 631, 'Lung Opacity + Nodule/Mass': 565, 'Lung Opacity + Pulmonary fibrosis': 545, 'Lung Opacity + Pleural effusion': 507, 'Aortic enlargement + Pleural effusion': 442, 'Aortic enlargement + Nodule/Mass': 437, 'Lung Opacity + Pleural thickening': 416, 'Nodule/Mass + Other lesion': 412, 'Other lesion + Pulmonary fibrosis': 412, 'Pleural effusion + Pulmonary fibrosis': 392, 'Cardiomegaly + Pleural thickening': 383, 'Aortic enlargement + Other lesion': 377, 'Infiltration + Lung Opacity': 359, 'Aortic enlargement + Lung Opacity': 342, 'Calcification + Pulmonary fibrosis': 327, 'Cardiomegaly + Other lesion': 326, 'Cardiomegaly + Pulmonary fibrosis': 325, 'Nodule/Mass + Pulmonary fibrosis': 314, 'Calcification + Nodul